In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import time

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\39404\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
reviews = pd.read_csv('./Reviews.csv')

In [4]:
reviews.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [5]:
reviews.shape

(568454, 10)

In [6]:
reviews.isnull().sum()

Id                         0
ProductId                  0
UserId                     0
ProfileName               16
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64

In [7]:
# reviews['Summary'].replace('',np.nan,inplace=True)
reviews.dropna(subset=['Summary', 'Text'],inplace=True)

In [8]:
reviews.isnull().sum()

Id                         0
ProductId                  0
UserId                     0
ProfileName               16
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                    0
Text                       0
dtype: int64

In [9]:
reviews.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [10]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are",
' american ':
        [
            'amerikan'
        ],

    ' adolf ':
        [
            'adolf'
        ],


    ' hitler ':
        [
            'hitler'
        ],

    ' fuck':
        [
            '(f)(u|[^a-z0-9 ])(c|[^a-z0-9 ])(k|[^a-z0-9 ])([^ ])*',
            '(f)([^a-z]*)(u)([^a-z]*)(c)([^a-z]*)(k)',
            ' f[!@#\$%\^\&\*]*u[!@#\$%\^&\*]*k', 'f u u c',
            '(f)(c|[^a-z ])(u|[^a-z ])(k)', r'f\*',
            'feck ', ' fux ', 'f\*\*', 
            'f\-ing', 'f\.u\.', 'f###', ' fu ', 'f@ck', 'f u c k', 'f uck', 'f ck','fuk', 'wtf','fucck','f cking'
        ],

    ' ass ':
        [
            '[^a-z]ass ', '[^a-z]azz ', 'arrse', ' arse ', '@\$\$'
                                                           '[^a-z]anus', ' a\*s\*s', '[^a-z]ass[^a-z ]',
            'a[@#\$%\^&\*][@#\$%\^&\*]', '[^a-z]anal ', 'a s s'
        ],

    ' asshole ':
        [
            ' a[s|z]*wipe', 'a[s|z]*[w]*h[o|0]+[l]*e', '@\$\$hole', 'ass hole'
        ],

    ' bitch ':
        [
            'b[w]*i[t]*ch', 'b!tch',
            'bi\+ch', 'b!\+ch', '(b)([^a-z]*)(i)([^a-z]*)(t)([^a-z]*)(c)([^a-z]*)(h)',
            'biatch', 'bi\*\*h', 'bytch', 'b i t c h','beetch'
        ],

    ' bastard ':
        [
            'ba[s|z]+t[e|a]+rd'
        ],

    ' transgender':
        [
            'transgender','trans gender'
        ],

    ' gay ':
        [
            'gay'
        ],

    ' cock ':
        [
            '[^a-z]cock', 'c0ck', '[^a-z]cok ', 'c0k', '[^a-z]cok[^aeiou]', ' cawk',
            '(c)([^a-z ])(o)([^a-z ]*)(c)([^a-z ]*)(k)', 'c o c k'
        ],

    ' dick ':
        [
            ' dick[^aeiou]', 'deek', 'd i c k','diick '
        ],

    ' suck ':
        [
            'sucker', '(s)([^a-z ]*)(u)([^a-z ]*)(c)([^a-z ]*)(k)', 'sucks', '5uck', 's u c k'
        ],

    ' cunt ':
        [
            'cunt', 'c u n t'
        ],

    ' bullshit ':
        [
            'bullsh\*t', 'bull\$hit','bs'
        ],

    ' homosexual':
        [
            'homo sexual','homosex'
        ],

    ' jerk ':
        [
            'jerk'
        ],

    ' idiot ':
        [
            'i[d]+io[t]+', '(i)([^a-z ]*)(d)([^a-z ]*)(i)([^a-z ]*)(o)([^a-z ]*)(t)', 'idiots', 'i d i o t'
        ],

    ' dumb ':
        [
            '(d)([^a-z ]*)(u)([^a-z ]*)(m)([^a-z ]*)(b)'
        ],

    ' shit ':
        [
            'shitty', '(s)([^a-z ]*)(h)([^a-z ]*)(i)([^a-z ]*)(t)', 'shite', '\$hit', 's h i t'
        ],

    ' shithole ':
        [
            'shythole','shit hole'
        ],

    ' retard ':
        [
            'returd', 'retad', 'retard', 'wiktard', 'wikitud'
        ],

    ' rape ':
        [
            ' raped'
        ],

    ' dumbass':
        [
            'dumb ass', 'dubass'
        ],

    ' asshead':
        [
            'butthead', 'ass head'
        ],

    ' sex ':
        [
            's3x', 'sexuality',
        ],


    ' nigger ':
        [
            'nigger', 'ni[g]+a', ' nigr ', 'negrito', 'niguh', 'n3gr', 'n i g g e r'
        ],

    ' shut the fuck up':
        [
            'stfu'
        ],

    ' pussy ':
        [
            'pussy[^c]', 'pusy', 'pussi[^l]', 'pusses'
        ],

    ' faggot ':
        [
            'faggot', ' fa[g]+[s]*[^a-z ]', 'fagot', 'f a g g o t', 'faggit',
            '(f)([^a-z ]*)(a)([^a-z ]*)([g]+)([^a-z ]*)(o)([^a-z ]*)(t)', 'fau[g]+ot', 'fae[g]+ot',
        ],

    ' motherfucker':
        [
            ' motha ', ' motha f', ' mother f', 'motherucker', 'mother fucker'
        ],

    ' whore ':
        [
            'wh\*\*\*', 'w h o r e'
        ],
    " White's " : 'white'
}

In [11]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'<br >', '', text)
    text = re.sub(r'\'', ' ', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text

In [12]:
clean_summaries = []
for summary in reviews.Summary:
    if not isinstance(summary, str):
        print(summary)
        continue
    clean_summaries.append(clean_text(summary, remove_stopwords=False))
print("Summaries are complete.")

clean_texts = []
for text in reviews.Text:
    clean_texts.append(clean_text(text))
print("Texts are complete.")

Summaries are complete.
Texts are complete.


In [13]:
# Inspect the cleaned summaries and texts to ensure they have been cleaned well
for i in range(15,20):
    print("Clean Review #",i+1)
    print("Summary: ",clean_summaries[i])
    print("Content: ",clean_texts[i])
    print()

Clean Review # 16
Summary:  lots of twizzlers  just what you expect 
Content:  daughter loves twizzlers shipment six pounds really hit spot exactly would expect six packages strawberry twizzlers

Clean Review # 17
Summary:  poor taste
Content:  love eating good watching tv looking movies sweet like transfer zip lock baggie stay fresh take time eating

Clean Review # 18
Summary:  love it 
Content:  satisfied twizzler purchase shared others enjoyed definitely ordering

Clean Review # 19
Summary:  great sweet candy 
Content:  twizzlers strawberry childhood favorite candy made lancaster pennsylvania candies inc one oldest confectionery firms united states subsidiary hershey company company established 1845 young smylie also make apple licorice twists green color blue raspberry licorice twists like all<br ><br >i keep dry cool place recommended put fridge according guinness book records longest licorice twist ever made measured 1 200 feet 370 weighted 100 pounds 45 kg made candies inc recor

In [14]:
def count_words(count_dict, text):
    '''Count the number of occurrences of each word in a set of text'''
    for sentence in text:
        for word in sentence.split():
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1

In [15]:
word_counts = {}

count_words(word_counts, clean_summaries)
count_words(word_counts, clean_texts)
            
print("Size of Vocabulary:", len(word_counts))

Size of Vocabulary: 132887


In [16]:
embeddings_index = {}
with open('./numberbatch-en-19.08.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding
print('Word embeddings:', len(embeddings_index))

Word embeddings: 516783


In [17]:
# Find the number of words that are missing from CN, and are used more than our threshold.
missing_words = 0
threshold = 20

for word, count in word_counts.items():
    if count > threshold:
        if word not in embeddings_index:
            missing_words += 1
            
missing_ratio = round(missing_words/len(word_counts),4)*100
            
print("Number of words missing from CN:", missing_words)
print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))

Number of words missing from CN: 3845
Percent of words that are missing from vocabulary: 2.8899999999999997%


In [18]:
# Limit the vocab that we will use to words that appear ≥ threshold or are in GloVe

#dictionary to convert words to integers
vocab_to_int = {} 

value = 0
for word, count in word_counts.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1

# Special tokens that will be added to our vocab
codes = ["<UNK>","<PAD>","<EOS>","<GO>"]   

# Add codes to vocab
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total number of unique words:", len(word_counts))
print("Number of words we will use:", len(vocab_to_int))
print("Percent of words we will use: {}%".format(usage_ratio))

Total number of unique words: 132887
Number of words we will use: 60433
Percent of words we will use: 45.48%


In [19]:
# Need to use 300 for embedding dimensions to match CN's vectors.
embedding_dim = 300
nb_words = len(vocab_to_int)

# Create matrix with default values of zero
word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        # If word not in CN, create a random embedding for it
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

# Check if value matches len(vocab_to_int)
print(len(word_embedding_matrix))

60433


In [20]:
def convert_to_ints(text, word_count, unk_count):
    '''Convert words in text to an integer(the index of words in vocabulary).
       If word is not in vocab_to_int, use UNK's integer.
       Total the number of words and UNKs.
       Add EOS token to the end of texts'''
    ints = []
    for sentence in text:
        sentence_ints = [vocab_to_int["<GO>"]]
        for word in sentence.split():
            word_count += 1
            if word in vocab_to_int:
                sentence_ints.append(vocab_to_int[word])
            else:
                sentence_ints.append(vocab_to_int["<UNK>"])
                unk_count += 1
       
        sentence_ints.append(vocab_to_int["<EOS>"])
        ints.append(sentence_ints)
    return ints, word_count, unk_count

In [21]:
# Apply convert_to_ints to clean_summaries and clean_texts
word_count = 0
unk_count = 0

int_summaries, word_count, unk_count = convert_to_ints(clean_summaries, word_count, unk_count)
int_texts, word_count, unk_count = convert_to_ints(clean_texts, word_count, unk_count)

unk_percent = round(unk_count/word_count,4)*100

print("Total number of words in headlines:", word_count)
print("Total number of UNKs in headlines:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))

Total number of words in headlines: 25680389
Total number of UNKs in headlines: 189833
Percent of words that are UNK: 0.74%


In [22]:
def create_lengths(text):
    '''Create a data frame of the sentence lengths from a text'''
    lengths = []
    for sentence in text:
        lengths.append(len(sentence))
    return pd.DataFrame(lengths, columns=['counts'])

In [23]:
lengths_summaries = create_lengths(int_summaries)
lengths_texts = create_lengths(int_texts)

print("Summaries:")
print(lengths_summaries.describe())
print()
print("Texts:")
print(lengths_texts.describe())

Summaries:
              counts
count  568427.000000
mean        6.181613
std         2.657912
min         2.000000
25%         4.000000
50%         6.000000
75%         7.000000
max        50.000000

Texts:
              counts
count  568427.000000
mean       42.996376
std        42.520534
min         2.000000
25%        19.000000
50%        30.000000
75%        51.000000
max      2086.000000


In [24]:
# Inspect the length of texts
print(np.percentile(lengths_texts.counts, 90))
print(np.percentile(lengths_texts.counts, 95))
print(np.percentile(lengths_texts.counts, 99))

85.0
116.0
208.0


In [25]:
# Inspect the length of summaries
print(np.percentile(lengths_summaries.counts, 90))
print(np.percentile(lengths_summaries.counts, 95))
print(np.percentile(lengths_summaries.counts, 99))

10.0
11.0
15.0


In [26]:
def unk_counter(sentence):
    '''Counts the number of time UNK appears in a sentence.'''
    unk_count = 0
    for word in sentence:
        if word == vocab_to_int["<UNK>"]:
            unk_count += 1
    return unk_count

In [27]:
# takes a long time  , this is normal

# Sort the summaries and texts by the length of the texts, shortest to longest
# Limit the length of summaries and texts based on the min and max ranges.
# Remove reviews that include too many UNKs

sorted_summaries = []
sorted_texts = []
max_text_length = 84
max_summary_length = 13
min_length = 2
unk_text_limit = 1
unk_summary_limit = 0

for length in range(min(lengths_texts.counts), max_text_length): 
    for count, words in enumerate(int_summaries):
        if (len(int_summaries[count]) >= min_length and
            len(int_summaries[count]) <= max_summary_length and
            len(int_texts[count]) >= min_length and
            unk_counter(int_summaries[count]) <= unk_summary_limit and
            unk_counter(int_texts[count]) <= unk_text_limit and
            length == len(int_texts[count])
           ):
            sorted_summaries.append(int_summaries[count])
            sorted_texts.append(int_texts[count])
        
# Compare lengths to ensure they match
print(len(sorted_summaries))
print(len(sorted_texts))

472585
472585


In [28]:
sorted_texts[-100:-1]

[[60432,
  32,
  174,
  32,
  3432,
  174,
  174,
  5864,
  485,
  20,
  68,
  746,
  4661,
  1871,
  2374,
  440,
  2273,
  68,
  1437,
  4661,
  1871,
  2374,
  440,
  68,
  380,
  0,
  690,
  176,
  475,
  174,
  3515,
  4229,
  174,
  2121,
  93,
  380,
  0,
  23548,
  23549,
  23566,
  174,
  1972,
  334,
  2717,
  7343,
  174,
  605,
  380,
  0,
  68,
  380,
  0,
  174,
  23548,
  23549,
  23576,
  174,
  0,
  541,
  565,
  87,
  114,
  13636,
  803,
  4986,
  117,
  4986,
  1267,
  174,
  1602,
  801,
  117,
  1043,
  23548,
  23549,
  23551,
  32,
  3432,
  174,
  509,
  1870,
  174,
  60431],
 [60432,
  39411,
  60429,
  23549,
  23583,
  1883,
  661,
  3655,
  33327,
  509,
  37665,
  19494,
  9440,
  2,
  489,
  62,
  2533,
  151,
  925,
  693,
  22059,
  61,
  259,
  2,
  3,
  396,
  1,
  478,
  9619,
  44237,
  44238,
  357,
  338,
  2940,
  193,
  189,
  6669,
  10994,
  190,
  4586,
  192,
  371,
  10457,
  1912,
  909,
  233,
  3,
  1096,
  4107,
  1010,
  25,
  303,
  

In [29]:
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import torch
import torch.nn.utils.rnn as rnn_utils
import torch.nn.functional as F
import torchvision
from torchvision import datasets, models, transforms
from torchvision.utils import make_grid
import torch.nn as nn
import matplotlib.pyplot as plt
import random 

C:\ProgramData\Anaconda3\envs\Text_Summarizer\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
# Subset the data for training
start = 200000
end = start + 300000
sorted_summaries_short = sorted_summaries[start:end]
sorted_texts_short = sorted_texts[start:end]
print("The shortest text length:", len(sorted_texts_short[0]))
print("The longest text length:",len(sorted_texts_short[-1]))

The shortest text length: 24
The longest text length: 83


In [31]:
" ".join([int_to_vocab[i] for i in sorted_texts_short[2]])

'<GO> makes perfect crab cake right recipe back packet made 3 instead 6 recommend broiled golden brown good g ms baltimore saying lot <EOS>'

In [32]:
train_size = int(len(sorted_texts_short) * 0.9)
val_size = len(sorted_texts_short) - train_size
zipped = list(zip(sorted_texts_short, sorted_summaries_short))
train_dataset, val_dataset = torch.utils.data.random_split(zipped, [train_size,val_size])
text_train, summ_train = zip(*train_dataset)
text_val, summ_val = zip(*val_dataset)


In [50]:
" ".join([int_to_vocab[i] for i in summ_train[3]])

'<GO> try one before you buy a case <EOS>'

In [51]:
" ".join([int_to_vocab[i] for i in text_train[3]])

'<GO> try one buy case regret spending 1 79 bottle like really going hate dumping 25 case crap hands beyond doubt worst commercially available drink least 10 years maybe stevia sweetener know barely choked quarter bottle pouring rest drain aftertaste abominable remotely qualify drink much less lemonade trying give zero stars website forcing give least one <EOS>'

In [42]:
[len(x)for x in text_train[:10]]

[30, 68, 59, 57, 42, 29, 39, 77, 31, 72]

In [52]:
def len_argsort(seq):
    return sorted(range(len(seq)), key=lambda x: len(seq[x]))

In [54]:
sorted_index = len_argsort(text_train)
text_train = [text_train[i] for i in sorted_index]
summ_train = [summ_train[i] for i in sorted_index]

In [61]:
[len(x)for x in text_train[-2:-1]] # 24 - 83

[83]

In [65]:
k = 2
print(" ".join([int_to_vocab[i] for i in text_train[k]]))
print(" ".join([int_to_vocab[i] for i in summ_train[k]]))

<GO> great tasting snack 130 calories 6 grams fat 1 serving bag 2 cups kids love idea oven baked natural snack feel good <EOS>
<GO> a sensible smart snack <EOS>


In [67]:
# 按索引划分batch，返回的每个batch中存的是index
def get_minibatches(n, minibatch_size, shuffle=True):
    idx_list = np.arange(0, n, minibatch_size)
    if shuffle:
        np.random.shuffle(idx_list)
    minibatches = []
    for idx in idx_list:
        minibatches.append(np.arange(idx, min(idx+minibatch_size, n)))
    return minibatches      # 这个会返回多批连着的bath_size个索引  
#get_minibatches(len(train_en), 32)

# 这个函数是在做数据预处理， 由于每个句子都不是一样长， 所以通过这个函数就可以把句子进行补齐， 不够长的在句子后面添加0
def prepare_data(seqs):
    lengths = [len(seq) for seq in seqs]    # 得到每个句子的长度
    n_samples = len(seqs)       # 得到一共有多少个句子
    max_len = np.max(lengths)              # 找出最大的句子长度
    
    x = np.zeros((n_samples, max_len)).astype('int32')    # 按照最大句子长度生成全0矩阵
    x_lengths = np.array(lengths).astype('int32')
    for idx, seq in enumerate(seqs):        # 把有句子的位置填充进去
        x[idx, :lengths[idx]] = seq
    return x, x_lengths      # x_mask

def gen_examples(en_sentences, cn_sentences, batch_size):
    minibatches = get_minibatches(len(en_sentences), batch_size)   # 得到batch个索引
    all_ex = []
    for minibatch in minibatches:   # 每批数据的索引
        mb_en_sentences = [en_sentences[t] for t in minibatch]   # 取数据
        mb_cn_sentences = [cn_sentences[t] for t in minibatch]  # 取数据
        mb_x, mb_x_len = prepare_data(mb_en_sentences) # 填充成一样的长度， 但是要记录一下句子的真实长度， 这个在后面输入网络的时候得用
        mb_y, mb_y_len = prepare_data(mb_cn_sentences)
        all_ex.append((mb_x, mb_x_len, mb_y, mb_y_len))
    return all_ex

batch_size = 64
train_data = gen_examples(text_train, summ_train, batch_size)   # 产生训练集
random.shuffle(train_data)
dev_data = gen_examples(text_val, summ_val, batch_size)   # 产生验证集


In [72]:
print(train_data[1][0].shape, train_data[1][1].shape, train_data[1][2].shape, train_data[1][3].shape)  

(64, 46) (64,) (64, 12) (64,)


In [98]:
class PlainEncoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, dropout=0.2):
        super(PlainEncoder, self).__init__()
        self.embed = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.GRU(hidden_size, hidden_size, batch_first=True)   # embedding layer最后输出的维度是hidden_size,相当于现在每个词的embedding维度等于hidden_size
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, lengths):  
        # 这里需要输入lengths， 因为每个句子是不一样长的，我们需要每个句子最后一个时间步的隐藏状态,所以需要知道句子有多长， x表示一个batch里面的句子 
        # 这里的x lengths 都是tensor， 调用tensor.sort, 返回的是排好的数组和index
        # 把batch里面的seq按照长度排序  
        sorted_len, sorted_idx = lengths.sort(0, descending=True)  #d_le sorted_len表示排好序的数组， sorted_index表示每个元素在原数组位置
        x_sorted = x[sorted_idx.long()]  # 将句子已经按照sorted_idx排序
        embedded = self.dropout(self.embed(x_sorted))   # [batch_size, seq_len, embed_size]
        #得到embedding， 每个句子的embedding维度等于hidden_size
        # 下面一段代码处理变长序列
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, sorted_len.long().cpu().data.numpy(), batch_first=True)    # 这里的data.numpy()是原来张量的克隆， 然后转成了numpy数组， 相当于clone().numpy()
        # 上面这句话之后， 会把变长序列的0都给去掉， 之前填充的字符都给压扁
        packed_out, hid = self.rnn(packed_embedded)    # 当输入为packed_embedding时，句子有多长，hidden state就有多长
        out, _ = nn.utils.rnn.pad_packed_sequence(packed_out, batch_first=True) # 将out再填充回去，便于后续处理（实际上后面直接抛弃了，有用的只是最后一个字的hidden state） 
        _, original_idx = sorted_idx.sort(0, descending=False)      # 恢复原来的顺序
        out = out[original_idx.long()].contiguous()  # 深拷贝rnn输出的tensor
        hid = hid[:, original_idx.long()].contiguous() 
        
        return out, hid[[-1]]   # 把最后一层的hid给拿出来  

In [99]:
class PlainDecoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, dropout=0.2):
        super(PlainDecoder, self).__init__()
        self.embed = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, y, y_lengths, hid):
        # y: [batch_size, y_length] y_length是最长句子的长度（每个句子都被padding成了这么长）
        # hid是encoder得到的最后一个字的hidden_state， 实际上包含了整个输入序列的信息
        sorted_len, sorted_idx = y_lengths.sort(0, descending=True)    # 依然是句子从长到短排序

        y_sorted = y[sorted_idx.long()] # 排序过程
        hid = hid[:, sorted_idx.long()] # 排序过程
        
        y_sorted = self.dropout(self.embed(y_sorted)) # batch_size, y_length, embed_size] 这里的embed_size实际上是等于hidden_size的
        
        pack_seq = nn.utils.rnn.pack_padded_sequence(y_sorted, sorted_len.long().cpu().data.numpy(), batch_first=True)
        out, hid = self.rnn(pack_seq, hid)   # decoder里面的初始state 就是encoder中得到的最后一个字的state
        unpacked, _ = nn.utils.rnn.pad_packed_sequence(out, batch_first=True)   # [batch, y_length, hidden_size] 恢复out的长度，（实际上是把packed_sequence 转回tensor）
        _, original_idx = sorted_idx.sort(0, descending=False) #恢复原来的句子顺序
        output_seq = unpacked[original_idx.long()].contiguous()  # [batch, y_length, hidden_size] 恢复顺序+深拷贝
        
        hid = hid[:, original_idx.long()].contiguous()   # [1， batch, hidden_size] decoder的hid_state其实没用，只需要输出即可。
        output = F.log_softmax(self.out(output_seq), -1)    
        #   self.out 是一个fully connected neural network,输出结果是[batch, y_length, vocab_size] 然后对最后一个维度 vocab_size进行归一化,那么现在output的每一步输出，都是一个vocab_size长度的vector，取其中数值最大的维度就得到了对应的单词
        
        return output, hid


In [100]:
class PlainSeq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(PlainSeq2Seq, self).__init__()
        self.encoder = encoder   
        self.decoder = decoder
        
    def forward(self, x, x_lengths, y, y_lengths):
        encoder_out, hid = self.encoder(x, x_lengths)  
        output, hid = self.decoder(y, y_lengths, hid) 
        return output, None

    def translate(self, x, x_lengths, y, max_length=10): # 这个是进来一个句子进行翻译, y是仅有一个起始字符<go>的decoder输入序列  max_length句子的最大长度
        encoder_out, hid = self.encoder(x, x_lengths)   # 将原来的summary进行解码
        preds = []
        batch_size = x.shape[0]
        attns = []
        for i in range(max_length):   
            output, hid = self.decoder(y, torch.ones(batch_size).long().to(y.device), hid=hid) # 编码，此时decoder的输入就只是<go>
            y = output.max(2)[1].view(batch_size, 1) #取dim=2 上的最大值所在的维度，即对应词在vocabulary上的index
            preds.append(y)
        
        return torch.cat(preds, 1), None


In [101]:
# masked cross entropy loss
class LanguageModelCriterion(nn.Module):
    def __init__(self):
        super(LanguageModelCriterion, self).__init__()
    
    def forward(self, input, target, mask):
        # input: [batch_size, seq_len, vocab_size]    每个单词的可能性
        input = input.contiguous().view(-1, input.size(2))   # [batch_size*seq_len-1, vocab_size] 将input的batch_size和seq_len合并，再计算loss
        target = target.contiguous().view(-1, 1)    #  [batch_size*seq_len-1, 1] # target里dim=1存的是正确summary 的index
        
        mask = mask.contiguous().view(-1, 1)   # [batch_size*seq_len-1, 1]
        output = -input.gather(1, target) * mask # 在每个vocab_size维度取正确单词的索引， 但是里面有很多是填充进去的， 所以mask去掉这些填充的
        # 这个其实在写一个NLloss ， 
        output = torch.sum(output) / torch.sum(mask)
        
        return output  # [batch_size*seq_len-1, 1]


In [102]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dropout = 0.2
hidden_size = 100
encoder = PlainEncoder(vocab_size=len(word_embedding_matrix), hidden_size=hidden_size, dropout=dropout)
decoder = PlainDecoder(vocab_size=len(word_embedding_matrix), hidden_size=hidden_size, dropout=dropout)

model = PlainSeq2Seq(encoder, decoder)
model = model.to(device)
loss_fn = LanguageModelCriterion().to(device)
optimizer = torch.optim.Adam(model.parameters())


In [103]:
# 定义训练和验证函数
def evaluate(model, data):
    model.eval()
    total_num_words = total_loss = 0.
    with torch.no_grad():
        for it, (mb_x, mb_x_len, mb_y, mb_y_len) in enumerate(data):
            mb_x = torch.from_numpy(mb_x).to(device).long()    # 这个是一个batch的英文句子 大小是[batch_size, seq_len]
            mb_x_len = torch.from_numpy(mb_x_len).to(device).long()    # 每个句子的长度
            mb_input = torch.from_numpy(mb_y[:, :-1]).to(device).long()  # decoder的输入，训练的时候是整个summary（带<GO>）
            mb_output = torch.from_numpy(mb_y[:, 1:]).to(device).long()   # 解码器那边的输出  [batch_size, seq_len-1]去掉<GO>
            mb_y_len = torch.from_numpy(mb_y_len-1).to(device).long()  # 这个减去1， 去掉初始符号 [batch_size, seq_len-1]
            mb_y_len[mb_y_len<=0] =  1   # 以防出错，防止trainset里面的summary长度被记为0（实际上不可能）
            
            mb_pred, attn = model(mb_x, mb_x_len, mb_input, mb_y_len)
            
            mb_out_mask = torch.arange(mb_y_len.max().item(), device=device)[None, :] < mb_y_len[:, None]  
            # [batch_size,最长句子的长度], 上面是bool类型， 下面是float类型， 只计算每个句子的有效部分， 填充的那部分去掉
            mb_out_mask = mb_out_mask.float()  # [batch_size, seq_len-1]  因为mb_y_len.max()就是seq_len-1
            
            loss = loss_fn(mb_pred, mb_output, mb_out_mask)
            
            num_words = torch.sum(mb_y_len).item()
            total_loss += loss.item() * num_words
            total_num_words += num_words
    print('Evaluation loss', total_loss / total_num_words)
  
def train(model, data, num_epochs=20):
    for epoch in range(num_epochs):
        model.train()
        total_num_words = total_loss = 0.
        for it, (mb_x, mb_x_len, mb_y, mb_y_len) in enumerate(data):
            mb_x = torch.from_numpy(mb_x).to(device).long()
            mb_x_len = torch.from_numpy(mb_x_len).to(device).long()
            mb_input = torch.from_numpy(mb_y[:, :-1]).to(device).long()
            mb_output = torch.from_numpy(mb_y[:, 1:]).to(device).long()
            mb_y_len = torch.from_numpy(mb_y_len-1).to(device).long()
            mb_y_len[mb_y_len<=0] = 1
            
            mb_pred, attn = model(mb_x, mb_x_len, mb_input, mb_y_len)
            
            mb_out_mask = torch.arange(mb_y_len.max().item(), device=device)[None, :] < mb_y_len[:, None]
            mb_out_mask = mb_out_mask.float()
            
            loss = loss_fn(mb_pred, mb_output, mb_out_mask)
            
            num_words = torch.sum(mb_y_len).item()
            total_loss += loss.item() * num_words
            total_num_words += num_words
            
            # 更新
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5.)     # 防止梯度爆炸
            optimizer.step()
            
            if it % 100 == 0:
                print('Epoch', epoch, 'iteration', it, 'loss', loss.item())
            
        print('Epoch', epoch, 'Training loss', total_loss / total_num_words)
        if epoch % 5 == 0:
            evaluate(model, dev_data)
        
# 训练
train(model, train_data, num_epochs=20)


Epoch 0 iteration 0 loss 11.039939880371094
Epoch 0 iteration 100 loss 5.816871643066406
Epoch 0 iteration 200 loss 5.8148627281188965
Epoch 0 iteration 300 loss 5.632275581359863
Epoch 0 iteration 400 loss 5.541488170623779
Epoch 0 iteration 500 loss 4.904646873474121
Epoch 0 iteration 600 loss 5.180214881896973
Epoch 0 iteration 700 loss 5.328367233276367
Epoch 0 iteration 800 loss 4.869574546813965
Epoch 0 iteration 900 loss 4.955080986022949
Epoch 0 iteration 1000 loss 4.714321136474609
Epoch 0 iteration 1100 loss 5.044029712677002
Epoch 0 iteration 1200 loss 4.967050552368164
Epoch 0 iteration 1300 loss 4.801717281341553
Epoch 0 iteration 1400 loss 4.750770568847656
Epoch 0 iteration 1500 loss 5.137588977813721
Epoch 0 iteration 1600 loss 4.992543697357178
Epoch 0 iteration 1700 loss 4.769374847412109
Epoch 0 iteration 1800 loss 4.797338485717773
Epoch 0 iteration 1900 loss 4.8608903884887695
Epoch 0 iteration 2000 loss 4.6577229499816895
Epoch 0 iteration 2100 loss 4.425401687622

Epoch 4 iteration 1800 loss 3.8560173511505127
Epoch 4 iteration 1900 loss 3.5874955654144287
Epoch 4 iteration 2000 loss 3.5854105949401855
Epoch 4 iteration 2100 loss 3.385143518447876
Epoch 4 iteration 2200 loss 3.255890130996704
Epoch 4 iteration 2300 loss 3.508324384689331
Epoch 4 iteration 2400 loss 3.9656217098236084
Epoch 4 iteration 2500 loss 3.3833677768707275
Epoch 4 iteration 2600 loss 3.7600343227386475
Epoch 4 iteration 2700 loss 3.6841373443603516
Epoch 4 iteration 2800 loss 3.466700553894043
Epoch 4 iteration 2900 loss 3.3992958068847656
Epoch 4 iteration 3000 loss 3.6358442306518555
Epoch 4 iteration 3100 loss 3.343780755996704
Epoch 4 iteration 3200 loss 3.4910190105438232
Epoch 4 iteration 3300 loss 3.869466781616211
Epoch 4 iteration 3400 loss 3.5682311058044434
Epoch 4 iteration 3500 loss 3.4095585346221924
Epoch 4 iteration 3600 loss 3.6633968353271484
Epoch 4 iteration 3700 loss 3.773346424102783
Epoch 4 iteration 3800 loss 3.5258872509002686
Epoch 4 Training los

Epoch 8 iteration 3500 loss 3.0568184852600098
Epoch 8 iteration 3600 loss 3.366919755935669
Epoch 8 iteration 3700 loss 3.4894683361053467
Epoch 8 iteration 3800 loss 3.239931344985962
Epoch 8 Training loss 3.2338542464883995
Epoch 9 iteration 0 loss 3.3509414196014404
Epoch 9 iteration 100 loss 2.862574338912964
Epoch 9 iteration 200 loss 3.235208034515381
Epoch 9 iteration 300 loss 3.2461354732513428
Epoch 9 iteration 400 loss 3.362874746322632
Epoch 9 iteration 500 loss 2.881356954574585
Epoch 9 iteration 600 loss 3.2277774810791016
Epoch 9 iteration 700 loss 3.239696502685547
Epoch 9 iteration 800 loss 2.9558279514312744
Epoch 9 iteration 900 loss 3.2365658283233643
Epoch 9 iteration 1000 loss 3.101370334625244
Epoch 9 iteration 1100 loss 3.353290319442749
Epoch 9 iteration 1200 loss 3.145198106765747
Epoch 9 iteration 1300 loss 3.0978240966796875
Epoch 9 iteration 1400 loss 3.105985641479492
Epoch 9 iteration 1500 loss 3.5732827186584473
Epoch 9 iteration 1600 loss 3.599020957946

Epoch 13 iteration 1000 loss 2.987788677215576
Epoch 13 iteration 1100 loss 3.121680974960327
Epoch 13 iteration 1200 loss 3.0104153156280518
Epoch 13 iteration 1300 loss 2.8705615997314453
Epoch 13 iteration 1400 loss 2.9396250247955322
Epoch 13 iteration 1500 loss 3.3500478267669678
Epoch 13 iteration 1600 loss 3.3611268997192383
Epoch 13 iteration 1700 loss 2.758031129837036
Epoch 13 iteration 1800 loss 3.419391632080078
Epoch 13 iteration 1900 loss 2.9655773639678955
Epoch 13 iteration 2000 loss 3.091520071029663
Epoch 13 iteration 2100 loss 2.7593531608581543
Epoch 13 iteration 2200 loss 2.81720232963562
Epoch 13 iteration 2300 loss 2.9621119499206543
Epoch 13 iteration 2400 loss 3.2374229431152344
Epoch 13 iteration 2500 loss 2.741091251373291
Epoch 13 iteration 2600 loss 3.191896677017212
Epoch 13 iteration 2700 loss 3.143864870071411
Epoch 13 iteration 2800 loss 2.842996835708618
Epoch 13 iteration 2900 loss 2.8645215034484863
Epoch 13 iteration 3000 loss 3.216550588607788
Epoc

Epoch 17 iteration 2400 loss 3.032316207885742
Epoch 17 iteration 2500 loss 2.6671292781829834
Epoch 17 iteration 2600 loss 3.1074442863464355
Epoch 17 iteration 2700 loss 3.0624160766601562
Epoch 17 iteration 2800 loss 2.6661651134490967
Epoch 17 iteration 2900 loss 2.845886707305908
Epoch 17 iteration 3000 loss 3.1623852252960205
Epoch 17 iteration 3100 loss 2.7457876205444336
Epoch 17 iteration 3200 loss 2.8433163166046143
Epoch 17 iteration 3300 loss 3.21230149269104
Epoch 17 iteration 3400 loss 2.7199862003326416
Epoch 17 iteration 3500 loss 2.842395067214966
Epoch 17 iteration 3600 loss 3.0075690746307373
Epoch 17 iteration 3700 loss 3.074907064437866
Epoch 17 iteration 3800 loss 2.948486804962158
Epoch 17 Training loss 2.8739083803393677
Epoch 18 iteration 0 loss 3.0211784839630127
Epoch 18 iteration 100 loss 2.499804735183716
Epoch 18 iteration 200 loss 2.8545114994049072
Epoch 18 iteration 300 loss 2.8422770500183105
Epoch 18 iteration 400 loss 2.9057490825653076
Epoch 18 iter

In [109]:
text_val[i]

[60432,
 4569,
 643,
 1871,
 10339,
 0,
 18409,
 7409,
 433,
 7140,
 2291,
 47450,
 643,
 25,
 258,
 1274,
 1525,
 32,
 2654,
 365,
 117,
 42567,
 643,
 780,
 1029,
 60431]

In [133]:
def translate_dev(i):
    text = " ".join([int_to_vocab[w] for w in text_val[i]])
    print(text)
    summary = " ".join([int_to_vocab[w] for w in summ_val[i]])
    print(summary)

    mb_x = torch.from_numpy(np.array(text_val[i]).reshape(1,-1)).long().to(device)
    mb_x_len = torch.from_numpy(np.array([len(text_val[i])])).long().to(device)
    # mb_y = torch.from_numpy(summ_val[i].reshape(1,-1)).long().to(device)
    # mb_y_len = torch.from_numpy([len(summ_val[i])]).long().to(device)
    bos = torch.Tensor([[vocab_to_int["<GO>"]]]).long().to(device)
    # 翻译时decoder的输入序列bos： [<GO>的index] 
    translation, attn = model.translate(mb_x,mb_x_len, bos)
    translation = [int_to_vocab[i] for i in translation.data.cpu().numpy().reshape(-1)]
    trans = []
    for word in translation:
        if word !="<EOS>":
            trans.append(word)
        else:
            break
    print(" ".join(trans))
    

In [134]:
for i in range(100,120):
    translate_dev(i)
    print()

<GO> lived wisconsin big fan choc moving overseas sent emails company asking get even act exporter answer came none borrow lewis carroll phrase pity <EOS>
<GO> chocolate a 5 but customer service a 0 <EOS>
delicious

<GO> bought product thought might make good backpacking food low fat calories really wanted like product like taste fact added sugar lot people seem like taste though unhappy gave try keep try using flavoring cooking <EOS>
<GO> does not taste like real peanut butter <EOS>
tastes like sugar

<GO> absolutely love different flavors coffee bundle delicious decent price honestly little surprised came big box k cups brown bag box like buy store small quirk probably buy k cups looking sellers <EOS>
<GO> yummy delicious but interesting packaging <EOS>
great coffee but price is too expensive

<GO> worst mix ever used saw reviews said bread comes like hard loaf used many pamelas products felt cannot products good sorry say right wrong choose product tried everything shorted bake time

In [137]:
torch.save(model.state_dict(), './model/checkpoint.pt')

In [140]:
model.load_state_dict(torch.load('./model/checkpoint.pt'))

<All keys matched successfully>

In [143]:
translate_dev(7)

<GO> bengal breed notorious digestive problems due fact hybrids cat chronic diarrhea since kitten mean non stop thousands dollars vet tests meds tried buying probiotics also massive fail poop stank bad even crystal litter thrown every couple days supposed last month three weeks first time kitty life normal cat poopies coat gleaming might even show thank royal canin <EOS>
<GO> thank god or this cat food <EOS>
great for cats


#### below are abandoned code

In [35]:
# class MyDataset(Dataset):
#   def __init__(self,sorted_texts,sorted_summaries, transform) -> None:
#     self.texts_set = sorted_texts
#     self.summaries_set = sorted_summaries
#     self.transform = transform
    
#     assert(len(self.texts_set)==len(self.summaries_set))

#   def __len__(self):
#     return len(self.texts_set)

#   def __getitem__(self, idx):

#     return (torch.Tensor(self.texts_set[idx]),torch.Tensor(self.summaries_set[idx]) )

In [36]:
# def collate_fn(data):
#     data.sort(key=lambda x: len(x[0]),reverse=True)
#     x = [i[0] for i in data]
#     x_lenth = [len(i)for i in x]
#     y = [i[1] for i in data]
#     y_lenth = [len(i)for i in y]
#     text = rnn_utils.pad_sequence(x, batch_first=True, padding_value=0)
#     summary = rnn_utils.pad_sequence(y, batch_first=True, padding_value=0)
#     return text.long(),x_lenth,summary.long(),y_lenth


In [37]:
# all_dataset = MyDataset(sorted_texts, sorted_summaries, None)
# train_size = int(len(all_dataset) * 0.9)
# validate_size = int(len(all_dataset) * 0.05)
# test_size = len(all_dataset)-train_size-validate_size
# train_dataset, validate_dataset, test_dataset = torch.utils.data.random_split(all_dataset, [train_size, validate_size, test_size])
# train_loader = DataLoader(train_dataset, batch_size=5000, shuffle=False, collate_fn=collate_fn)
# validate_loader = DataLoader(validate_dataset, batch_size=500, shuffle=False, collate_fn=collate_fn)
# test_loader = DataLoader(test_dataset, batch_size=500, shuffle=False, collate_fn=collate_fn)


# print(len(train_loader))
# print(len(validate_loader))
# print(len(test_loader))


In [38]:
# class Seq2seq(nn.Module):
#     def __init__(self, input_size, hidden_size = 256, num_layers = 2, drop_pro = 0.25):
#         super(Seq2seq, self).__init__()
#         self.encoding_layer = nn.LSTM(
#             input_size=input_size,
#             hidden_size=hidden_size,
#             num_layers=num_layers,
#             batch_first=True,
#             dropout = drop_pro,
#             bidirectional = True
#         )
#         self.decoding_layer = nn.LSTM(
#             input_size=input_size,
#             hidden_size=hidden_size,
#             num_layers=num_layers,
#             batch_first=True,
#             dropout = drop_pro,
#         )
        


#     def forward(self, x, y):
#          output, output_state = self.encoding_layer(x)
#          print(output)  
         
#        out, hidden_prev = self.rnn(x, hidden_prev)
#        # [b, seq, h]
#        out = out.view(-1, hidden_size)
#        out = self.linear(out)#[seq,h] => [seq,3]
#        out = out.unsqueeze(dim=0)  # => [1,seq,3]
#        return out, hidden_prev
    
  
        


In [39]:
# epochs = 20
# batch_size = 16
# rnn_size = 256 
# num_layers = 2
# learning_rate = 0.005
# drop_out = 0.25

# def fit(train_loader,epochs = 1, lr=0.001, drop_out=0.25):
#     embedding = torch.Tensor(word_embedding_matrix)
#     model = Seq2seq(embedding_dim,rnn_size, num_layers ,drop_out)
#     for epoch in range(epochs): 
#         train_losses = []
#         for batch in train_loader:
#             text,t_len,summary,s_len = batch
#             text = F.embedding(text, embedding)
#             text_pack = rnn_utils.pack_padded_sequence(text, t_len, batch_first=True, enforce_sorted = False)
            
# #Remove the last word id from each batch and concat the <GO> to the begining of each batch'''
#             go_pad = torch.full([summary.size()[0],1],  vocab_to_int['<GO>'])
#             summary = torch.cat((go_pad,summary),1)
#             summary = F.embedding(summary, embedding)
#             summary_pack = rnn_utils.pack_padded_sequence(summary, s_len, batch_first=True,  enforce_sorted = False)
#             model(text_pack, summary_pack)

#             break
#         break

In [40]:
# fit(train_loader)